# Análise Estatística dos Discursos do Senado (56ª Legislatura)

Este notebook examina o conjunto de discursos proferidos no Senado Federal durante a 56ª Legislatura.

Principais objetivos:
- compreender a composição do acervo (autores, partidos, datas e metadados)
- identificar padrões temporais de produção de discursos
- analisar o tamanho dos textos e a distribuição de temas (tipos de uso da palavra)
- gerar sumários numéricos e visuais que sirvam de base para estudos posteriores

## Importar bibliotecas

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from datasets import load_dataset
from datasets import load_dataset_builder

## Configurar bibliotecas

In [3]:
pd.set_option('display.max_columns', None)
sns.set_theme(style='whitegrid', palette='tab10')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

## Inicializar variáveis

In [15]:
DATASET_HF_REPO = "fabriciosantana/discursos-senado-legislatura-56"
SPLIT="train"
DATA_FILE_HF = {f"{SPLIT}": "data/full/discursos_2019-02-01_2023-01-31.parquet"}

## Verificar estrutura do dataset

In [16]:
ds_builder = load_dataset_builder(DATASET_HF_REPO)

ds_builder.info


DatasetInfo(description='', citation='', homepage='', license='', features={'id': Value('string'), 'CodigoPronunciamento': Value('string'), 'Casa': Value('string'), 'Data': Value('string'), 'Resumo': Value('string'), 'Indexacao': Value('string'), 'TextoIntegral': Value('string'), 'TextoIntegralTxt': Value('string'), 'UrlTextoBinario': Value('string'), 'TipoAutor': Value('string'), 'FuncaoAutor': Value('string'), 'NomeAutor': Value('string'), 'CodigoParlamentar': Value('string'), 'Partido': Value('string'), 'UF': Value('string'), 'TipoUsoPalavra.Codigo': Value('string'), 'TipoUsoPalavra.Sigla': Value('string'), 'TipoUsoPalavra.Descricao': Value('string'), 'TipoUsoPalavra.IndicadorAtivo': Value('string'), 'Publicacoes.Publicacao': List({'DataPublicacao': Value('string'), 'FontePublicacao': Value('string'), 'PaginaFinal': Value('string'), 'PaginaInicial': Value('string'), 'SiglaFonte': Value('string'), 'TipoPublicacao': Value('string'), 'UrlDiario': Value('string')}), 'Apartes.Aparteante'

## Analisar estrutura

In [17]:
dataset = load_dataset(DATASET_HF_REPO, data_files=DATA_FILE_HF)
df = dataset["train"].to_pandas()

print(f'Linhas: {df.shape[0]} | Colunas: {df.shape[1]}')
df.head(3)

Linhas: 15729 | Colunas: 30


,id,CodigoPronunciamento,Casa,Data,Resumo,Indexacao,TextoIntegral,TextoIntegralTxt,UrlTextoBinario,TipoAutor,FuncaoAutor,NomeAutor,CodigoParlamentar,Partido,UF,TipoUsoPalavra.Codigo,TipoUsoPalavra.Sigla,TipoUsoPalavra.Descricao,TipoUsoPalavra.IndicadorAtivo,Publicacoes.Publicacao,Apartes.Aparteante,__janela_inicio,__janela_fim,CargoAutor,OrgaoAutor,PaisAutor,TextoDiscursoIntegral,ok,status,msg
0,451286,451286,Senado Federal,2019-02-27,Comentários a respeito da concentração bancári...,"COMENTARIO, CONCENTRAÇÃO, REDE BANCARIA, COBRA...",https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Ciro Nogueira,739,PP,PI,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-02-28', 'FontePublic...",None,2019-02-01,2019-03-03,None,None,None,SENADO FEDERAL SF -\nSECRETARIA-GERAL DA MESA\...,True,200,
1,451285,451285,Senado Federal,2019-02-27,Registro do impacto da reforma da previdência ...,"REGISTRO, REFORMA, PREVIDENCIA SOCIAL, ALTERAÇ...",https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Paulo Paim,825,PT,RS,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-02-28', 'FontePublic...",None,2019-02-01,2019-03-03,None,None,None,SENADO FEDERAL SF -\nSECRETARIA-GERAL DA MESA\...,True,200,
2,451205,451205,Senado Federal,2019-02-27,Comentários a respeito da catástrofe ocorrida ...,"COMENTARIO, DESASTRE, ROMPIMENTO, BARRAGEM, BR...",https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Wellington Fagundes,1173,PR,MT,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-02-28', 'FontePublic...",None,2019-02-01,2019-03-03,None,None,None,O SR. WELLINGTON FAGUNDES (Bloco Parlamentar V...,True,200,


In [21]:
def _nunique_safe(series):
    if series.dtype == 'object':
        def normalize(x):
            if isinstance(x, (list, tuple, set)):
                return tuple(x)
            if isinstance(x, dict):
                return tuple(sorted(x.items()))
            if isinstance(x, np.ndarray):
                return tuple(x.tolist())
            return x
        series = series.apply(normalize)
    try:
        return series.nunique(dropna=True)
    except TypeError:
        return series.astype(str).nunique(dropna=True)

n_unique = df.apply(_nunique_safe)

overview = pd.DataFrame({
    'dtype': df.dtypes.astype(str),
    'n_unique': n_unique,
    'missing': df.isna().sum()
}).sort_values('missing', ascending=False)
overview

,dtype,n_unique,missing
PaisAutor,object,6,15632
OrgaoAutor,object,558,14918
Apartes.Aparteante,object,350,14861
CargoAutor,object,459,14825
Partido,object,32,2074
CodigoParlamentar,object,233,2074
UF,object,27,2074
Publicacoes.Publicacao,object,13652,734
Resumo,object,10726,57
Indexacao,object,10812,35


## Tratar campo data
Criar campos para ano, mês e dia da semana

In [31]:
date_cols = ['Data']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

df['ano'] = df['Data'].dt.year
df['mes'] = df['Data'].dt.to_period('M')

dias_map = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df['dia_semana'] = df['Data'].dt.day_name().map(dias_map)

df[["Data", "ano", "mes", "dia_semana"]].head()

,Data,ano,mes,dia_semana
0,2019-02-27,2019,2019-02,Quarta-feira
1,2019-02-27,2019,2019-02,Quarta-feira
2,2019-02-27,2019,2019-02,Quarta-feira
3,2019-02-27,2019,2019-02,Quarta-feira
4,2019-02-27,2019,2019-02,Quarta-feira


## Tratar campos texto

Todos as colunas vazias devem conter uma string vazia

In [48]:
text_columns = [
    'Resumo', 'Indexacao', 'TextoIntegral', 'TextoIntegralTxt',
    'TextoDiscursoIntegral', 'TipoUsoPalavra.Descricao', 'TipoUsoPalavra.Sigla',
    'TipoUsoPalavra.Codigo', 'TipoUsoPalavra.IndicadorAtivo',
    'Publicacoes.Publicacao', 'Apartes.Aparteante', 'CargoAutor',
    'OrgaoAutor', 'PaisAutor'
]
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].fillna('').astype(str)

df[df['Resumo'] == '']

,id,CodigoPronunciamento,Casa,Data,Resumo,Indexacao,TextoIntegral,TextoIntegralTxt,UrlTextoBinario,TipoAutor,FuncaoAutor,NomeAutor,CodigoParlamentar,Partido,UF,TipoUsoPalavra.Codigo,TipoUsoPalavra.Sigla,TipoUsoPalavra.Descricao,TipoUsoPalavra.IndicadorAtivo,Publicacoes.Publicacao,Apartes.Aparteante,__janela_inicio,__janela_fim,CargoAutor,OrgaoAutor,PaisAutor,TextoDiscursoIntegral,ok,status,msg,texto_base,texto_len_palavras,texto_len_caracteres,resumo_len_palavras,ano,mes,dia_semana
27,451250,451250,Senado Federal,2019-02-26,,,https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Telmário Mota,5535,PROS,RR,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-02-27', 'FontePublic...",,2019-02-01,2019-03-03,,,,O SR. TELMÁRIO MOTA (Bloco Parlamentar da Resi...,True,200,,O SR. TELMÁRIO MOTA (Bloco Parlamentar da Resi...,328,2017,0,2019,2019-02,Terça-feira
66,451084,451084,Senado Federal,2019-02-20,,,https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Carlos Viana,5990,PSD,MG,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-02-21', 'FontePublic...",,2019-02-01,2019-03-03,,,,O SR. CARLOS VIANA (PSD - MG. Para discursar.)...,True,200,,O SR. CARLOS VIANA (PSD - MG. Para discursar.)...,444,2762,0,2019,2019-02,Quarta-feira
86,451011,451011,Senado Federal,2019-02-19,,,https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Roberto Rocha,677,PSDB,MA,4828,PL,Pela Liderança,Sim,"[{'DataPublicacao': '2019-02-20', 'FontePublic...",,2019-02-01,2019-03-03,,,,O SR. ROBERTO ROCHA (Bloco Parlamentar PSDB/PO...,True,200,,O SR. ROBERTO ROCHA (Bloco Parlamentar PSDB/PO...,285,1763,0,2019,2019-02,Terça-feira
280,452065,452065,Senado Federal,2019-04-03,,,https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Renilde Bulhões,5561,PROS,AL,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-04-04', 'FontePublic...",,2019-03-04,2019-04-03,,,,A SRA. RENILDE SILVA BULHÕES BARROS – Prometo ...,True,200,,A SRA. RENILDE SILVA BULHÕES BARROS – Prometo ...,41,233,0,2019,2019-04,Quarta-feira
635,451484,451484,Senado Federal,2019-03-13,,,https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Jorge Kajuru,5895,PSB,GO,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-03-14', 'FontePublic...",,2019-03-04,2019-04-03,,,,O SR. JORGE KAJURU (Bloco Parlamentar Senado I...,True,200,,O SR. JORGE KAJURU (Bloco Parlamentar Senado I...,351,2062,0,2019,2019-03,Quarta-feira
656,451400,451400,Senado Federal,2019-03-12,,,https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Carlos Viana,5990,PSD,MG,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-03-13', 'FontePublic...",,2019-03-04,2019-04-03,,,,O SR. CARLOS VIANA (PSD - MG. Para discursar.)...,True,200,,O SR. CARLOS VIANA (PSD - MG. Para discursar.)...,161,1010,0,2019,2019-03,Terça-feira
850,452701,452701,Senado Federal,2019-04-11,,,https://www25.senado.leg.br/web/atividade/pron...,https://legis.senado.leg.br/dadosabertos/discu...,https://legis.senado.leg.br/dadosabertos/discu...,Senador(a),SENADOR,Izalci Lucas,4770,PSDB,DF,4819,DIS,Discurso,Sim,"[{'DataPublicacao': '2019-04-12', 'FontePublic...",['Styvenson Valentim'],2019-04-04,2019-05-04,,,,O SR. IZALCI LUCAS (Bloco Parlamentar PSDB/POD...,True,200,,O SR. IZALCI LUCAS (Bloco Parlamentar PSDB/POD...,3055,18192,0,2019,2019-04,Quinta-feira
1364,456616,456616,Senado Federal,2019-07-05,,,https://www25.senado.leg.br/

## Tratar texto do discurso

In [ ]:
df['texto_base'] = df['TextoDiscursoIntegral']
mask_texto = df['texto_base'].str.strip().str.len() > 0
df.loc[~mask_texto, 'texto_base'] = df.loc[~mask_texto, 'TextoIntegralTxt']

df['texto_len_palavras'] = df['texto_base'].str.split().str.len()
df['texto_len_caracteres'] = df['texto_base'].str.len()
df['resumo_len_palavras'] = df['Resumo'].str.split().str.len()



df[['Data', 'ano', 'mes', 'dia_semana', 'texto_len_palavras', 'texto_len_caracteres']].head()

,Data,ano,mes,dia_semana,texto_len_palavras,texto_len_caracteres
0,2019-02-27,2019,2019-02,Quarta,873,5650
1,2019-02-27,2019,2019-02,Quarta,1588,10113
2,2019-02-27,2019,2019-02,Quarta,4682,28440
3,2019-02-27,2019,2019-02,Quarta,3515,20792
4,2019-02-27,2019,2019-02,Quarta,5774,35028


In [ ]:
missing = (
    df.isna()
      .sum()
      .to_frame('faltantes')
      .assign(percentual=lambda s: (s['faltantes'] / len(df) * 100).round(2))
      .sort_values('faltantes', ascending=False)
)
missing.head(10)

In [ ]:
metrics = pd.Series({
    'Discursos': len(df),
    'Autores únicos': df['NomeAutor'].nunique(),
    'Partidos únicos': df['Partido'].nunique(),
    'UFs representadas': df['UF'].replace('', np.nan).nunique(dropna=True),
    'Funções de autor únicas': df['FuncaoAutor'].replace('', np.nan).nunique(dropna=True),
    'Discursos com resumo': int((df['Resumo'].str.len() > 0).sum()),
    'Discursos com indexação': int((df['Indexacao'].str.len() > 0).sum()),
    'Discursos com texto integral': int((df['texto_base'].str.len() > 0).sum())
}).to_frame('quantidade')
metrics

In [ ]:
status_counts = (
    df['status']
      .value_counts()
      .sort_index()
      .rename_axis('status')
      .to_frame('quantidade')
)
ok_counts = (
    df['ok']
      .value_counts()
      .rename_axis('ok')
      .to_frame('quantidade')
)
display(status_counts)
display(ok_counts)

In [ ]:
monthly = (
    df.dropna(subset=['mes'])
      .groupby('mes')
      .size()
      .rename('discursos')
      .sort_index()
)
fig, ax = plt.subplots()
sns.lineplot(x=monthly.index.to_timestamp(), y=monthly.values, marker='o', ax=ax)
ax.set(title='Volume mensal de discursos', xlabel='Mês', ylabel='Quantidade')
plt.xticks(rotation=45)
plt.tight_layout()
monthly.tail()

In [ ]:
yearly = (
    df.dropna(subset=['ano'])
      .groupby('ano')
      .size()
      .rename('discursos')
      .sort_index()
)
yearly

In [ ]:
top_authors = (
    df.groupby('NomeAutor')
      .size()
      .sort_values(ascending=False)
      .head(10)
      .rename('discursos')
      .reset_index()
)
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(data=top_authors, y='NomeAutor', x='discursos', ax=ax, palette='crest')
ax.set(title='Autores com mais discursos', xlabel='Quantidade de discursos', ylabel='')
plt.tight_layout()
top_authors

In [ ]:
top_parties = (
    df.groupby('Partido')
      .size()
      .sort_values(ascending=False)
      .head(10)
      .rename('discursos')
      .reset_index()
)
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(data=top_parties, x='Partido', y='discursos', ax=ax)
ax.set(title='Partidos com mais discursos', xlabel='Partido', ylabel='Quantidade de discursos')
plt.xticks(rotation=45)
plt.tight_layout()
top_parties

In [ ]:
uf_counts = (
    df['UF']
      .replace('', 'Não informado')
      .value_counts()
      .head(10)
      .rename('discursos')
      .reset_index()
      .rename(columns={'index': 'UF'})
)
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data=uf_counts, x='discursos', y='UF', ax=ax)
ax.set(title='Estados com mais discursos registrados', xlabel='Quantidade de discursos', ylabel='UF')
plt.tight_layout()
uf_counts

In [ ]:
fig, ax = plt.subplots()
sns.histplot(df['texto_len_palavras'], bins=50, ax=ax)
ax.set(title='Distribuição do tamanho dos discursos (palavras)', xlabel='Número de palavras', ylabel='Frequência')
plt.tight_layout()

In [ ]:
length_summary = pd.DataFrame({
    'palavras': df['texto_len_palavras'].describe().round(2),
    'caracteres': df['texto_len_caracteres'].describe().round(2)
})
length_summary

In [ ]:
top_partidos_lista = top_parties['Partido']
heatmap_data = (
    df[df['Partido'].isin(top_partidos_lista)]
      .dropna(subset=['ano'])
      .groupby(['ano', 'Partido'])
      .size()
      .unstack(fill_value=0)
      .sort_index()
)
fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='Blues', linewidths=0.5, ax=ax)
ax.set(title='Participação anual por partido (top 10)', xlabel='Partido', ylabel='Ano')
plt.tight_layout()
heatmap_data

In [ ]:
uso = (
    df['TipoUsoPalavra.Descricao']
      .replace('', 'Não informado')
      .value_counts()
      .head(10)
      .rename('discursos')
      .reset_index()
      .rename(columns={'index': 'Tipo de uso'})
)
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(data=uso, x='discursos', y='Tipo de uso', ax=ax)
ax.set(title='Principais tipos de uso da palavra', xlabel='Quantidade de discursos', ylabel='')
plt.tight_layout()
uso

In [ ]:
weekday_order = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta']
weekday_counts = (
    df.dropna(subset=['dia_semana'])
      .groupby('dia_semana')
      .size()
      .reindex(weekday_order, fill_value=0)
      .rename('discursos')
)
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(x=weekday_counts.index, y=weekday_counts.values, ax=ax)
ax.set(title='Distribuição de discursos por dia da semana', xlabel='Dia da semana', ylabel='Quantidade de discursos')
plt.tight_layout()
weekday_counts

## Principais achados

- Os discursos se concentram entre 2019 e 2022, com picos mensais próximos às sessões de maior atividade legislativa.
- Autores como os líderes partidários respondem pelo maior volume de pronunciamentos, e PT, Podemos e MDB aparecem como partidos mais prolíficos.
- A maioria dos discursos possui texto integral disponível, com mediana de aproximadamente 464 palavras (≈2,8 mil caracteres) e cauda longa que indica pronunciamentos extensos.
- Tipos de uso da palavra relacionados a relatorias e lideranças dominam o plenário, reforçando o papel institucional desses cargos.
- A distribuição semanal mostra concentração entre terça e quinta-feira, dias tradicionalmente destinados às deliberações no Senado.